In [11]:
from datetime import datetime as dt
import uuid 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import url_preparation as up

In [3]:
in_path = './../../../'
dataset = pd.read_csv(
    in_path+'splunk_data_180918_telenor_processed.txt',  
    encoding="ISO-8859-1", 
    dtype={
        "user_id": int, 
        "visit_id": int, 
        "sequence": int, 
        "start_time":object, 
        "event_duration":float,
        "url":str, 
        "action":str, 
        "country":str,
        "user_client":str,
        "user_client_family":str,
        "user_experience":str,
        "user_os":str,
        "apdex_user_experience":str,
        "bounce_rate":float,
        "session_duration":float
    }
)

In [4]:
t = dataset
t.columns = t.columns.str.replace('min_bedrift_event.','')
t = t[~t.action.isnull()]

# drop NaN actions or urls
t = t.dropna(axis='rows', how='any',subset=['url', 'action'])
#t = t.reset_index()

In [5]:
len(t.index)

3605250

In [9]:
print('original number of unique urls: %d' % len(set(t.url)))

original number of unique urls: 537339


In [39]:
 api_keywords = pd.read_csv('API_keywords.txt')['keywords']

In [46]:
def prepare_urls(df):
    df['url_cleaned'] = df['url']
    for k in api_keywords:
        df['url_cleaned'] = df['url_cleaned'].apply(lambda x: k if k in x else x)   
    return df

In [47]:
s = prepare_urls(t)
print('number of unique urls after replacement: %d' % len(set(s.url_cleaned)))

number of unique urls after replacement: 2168


In [44]:
s['url_cleaned'] = s['url_cleaned'].apply(lambda x: x if x in api_keywords.values else 'other_url')    
print('number of replaced unique urls after replacement: %d' % len(set(s.url_cleaned)))

number of replaced unique urls after replacement: 30


In [45]:
s[(s['url_cleaned']!='other_url')].shape[0]/s.shape[0]

0.8212592746688857

In [35]:
url_set = set(s['url_cleaned'])
url_set

{'/accessdenied',
 '/admins',
 '/fixed/accounts',
 '/fixed/locations',
 '/fixed/subscriptions',
 '/globalsearch',
 '/go-to-telenor.no',
 '/messages',
 '/mobile/accounts',
 '/mobile/agreements',
 '/mobile/agreementtypes',
 '/mobile/companies',
 '/mobile/newAgreement',
 '/mobile/newsubscription',
 '/mobile/orderhistory',
 '/mobile/recommendations',
 '/mobile/simcards',
 '/mobile/subscriptions',
 '/newadmin',
 '/others/api-services',
 '/others/bedrift',
 '/others/cloudservices',
 '/others/ordercases',
 '/others/tickets',
 '/reports/download',
 '/reports/order',
 '/reports/reporttype',
 '/unpaidinvoices',
 '/useragreement',
 'other_url'}

In [49]:
api_keywords = pd.read_csv('API_keywords.txt')['keywords']
api_keywords

0              /reports/order
1           /reports/download
2         /reports/reporttype
3                     /admins
4               /globalsearch
5       /mobile/subscriptions
6     /mobile/newsubscription
7            /mobile/simcards
8            /mobile/accounts
9           /mobile/companies
10     /mobile/agreementtypes
11          /mobile/companies
12         /mobile/agreements
13       /mobile/newAgreement
14       /mobile/orderhistory
15    /mobile/recommendations
16            /unpaidinvoices
17       /fixed/subscriptions
18            /fixed/accounts
19           /fixed/locations
20                  /newadmin
21            /others/tickets
22         /others/ordercases
23       /others/api-services
24      /others/cloudservices
25            /others/bedrift
26                  /messages
27          /go-to-telenor.no
28             /useragreement
29              /accessdenied
Name: keywords, dtype: object

In [38]:
s.url_cleaned.tail(10).values

array(['/mobile/newsubscription', '/mobile/subscriptions',
       '/mobile/accounts', '/mobile/subscriptions', '/globalsearch',
       '/newadmin', '/mobile/accounts', '/mobile/subscriptions',
       'other_url', '/newadmin'], dtype=object)

In [50]:
with open("url_api_keywords.tex", "w") as f:
    f.write(api_keywords.to_latex(index=False))